# K-Means Clustering for Route Recommendations

**Purpose:** Cluster all routes into k groups (e.g., k=10) to enable "curveball" recommendations from different clusters.

**Goal:** 
- Find optimal k (number of clusters)
- Cluster routes by similarity
- Profile each cluster to enable manual labeling (e.g., 'long mountain ride', 'short city ride')
- Save model for production use

**Use Case:** When a user uploads a route, serve:
- 5 nearest neighbors (KNN) from same/similar characteristics
- 1 "curveball" from a different cluster

## 1. Setup and Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score
)
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# Load the engineered data
df = pd.read_csv("../../api_faf/app/Data_Engineered.csv")
print(f"✅ Loaded {len(df):,} routes")
print(f"   Columns: {df.shape[1]}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Check for missing values
missing = df.isnull().sum()
if missing.sum() > 0:
    print("⚠️  Missing values found:")
    print(missing[missing > 0])
else:
    print("✅ No missing values")

## 2. Prepare Features (Same as KNN Model)

Use the same scaling approach as the KNN model for consistency.

In [ ]:
# Create feature matrix (exclude id, name, region)
X = df.drop(['id', 'name', 'region'], axis=1)
feature_cols = X.columns.tolist()

print(f"✅ Feature matrix created: {X.shape}")
print(f"\n📊 Features ({len(feature_cols)}):")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i:2d}. {col}")

In [ ]:
# Apply same scaling as KNN training
scaler = ColumnTransformer(transformers=[
    ('standard', StandardScaler(), [
        'distance_m', 'duration_s', 'ascent_m', 'descent_m', 
        'Turn_Density', 'steps', 'turns'
    ]),
    ('minmax', MinMaxScaler(), [
        'Cycleway', 'on_road', 'off_road', 'Gravel_Tracks', 'Paved_Paths', 
        'Other', 'Unknown Surface', 'Paved_Road', 'Pedestrian', 'Unknown_Way', 
        'Cycle Track', 'Main Road', 'Steep Section', 'Moderate Section', 
        'Flat Section', 'Downhill Section', 'Steep Downhill Section'
    ]),
], remainder='passthrough')

X_scaled = scaler.fit_transform(X)
print(f"✅ Features scaled: {X_scaled.shape}")
print(f"   StandardScaler: 7 features (distance, duration, elevation, turns)")
print(f"   MinMaxScaler: 17 features (surface types, gradients)")

## 3. K-Means Parameter Search

Test different values of k and initialization methods to find optimal clustering.

In [ ]:
# Define parameter grid
k_values = range(3, 21)  # Test k from 3 to 20
init_methods = ['k-means++', 'random']  # Initialization strategies
n_init = 20  # Number of times to run k-means with different seeds

print(f"🔍 Parameter Search Configuration:")
print(f"   k values: {list(k_values)}")
print(f"   Init methods: {init_methods}")
print(f"   n_init (runs per config): {n_init}")
print(f"\n⏳ This will test {len(k_values) * len(init_methods)} configurations...")

### 3.1 Evaluation Metrics

We'll use multiple metrics to evaluate clustering quality:

1. **Inertia (Within-cluster sum of squares)**: Lower is better. Shows how tight clusters are.
2. **Silhouette Score**: Range [-1, 1]. Higher is better. Measures how similar points are to their own cluster vs other clusters.
3. **Davies-Bouldin Index**: Lower is better. Measures average similarity between clusters.
4. **Calinski-Harabasz Score**: Higher is better. Ratio of between-cluster to within-cluster dispersion.

In [ ]:
# Run parameter search
results = []

for k in k_values:
    for init in init_methods:
        print(f"Testing k={k:2d}, init='{init}'...", end=' ')
        
        # Fit k-means
        kmeans = KMeans(
            n_clusters=k,
            init=init,
            n_init=n_init,
            max_iter=300,
            random_state=42
        )
        labels = kmeans.fit_predict(X_scaled)
        
        # Calculate metrics
        inertia = kmeans.inertia_
        silhouette = silhouette_score(X_scaled, labels, sample_size=10000)  # Sample for speed
        davies_bouldin = davies_bouldin_score(X_scaled, labels)
        calinski = calinski_harabasz_score(X_scaled, labels)
        
        results.append({
            'k': k,
            'init': init,
            'inertia': inertia,
            'silhouette': silhouette,
            'davies_bouldin': davies_bouldin,
            'calinski_harabasz': calinski
        })
        
        print(f"✓ Silhouette: {silhouette:.4f}")

# Convert to DataFrame
results_df = pd.DataFrame(results)
print(f"\n✅ Parameter search complete! Tested {len(results_df)} configurations.")

In [ ]:
# Display results sorted by different metrics
print("=" * 100)
print("TOP 10 CONFIGURATIONS BY SILHOUETTE SCORE (higher is better)")
print("=" * 100)
print(results_df.sort_values('silhouette', ascending=False).head(10))

print("\n" + "=" * 100)
print("TOP 10 CONFIGURATIONS BY DAVIES-BOULDIN INDEX (lower is better)")
print("=" * 100)
print(results_df.sort_values('davies_bouldin', ascending=True).head(10))

print("\n" + "=" * 100)
print("TOP 10 CONFIGURATIONS BY CALINSKI-HARABASZ SCORE (higher is better)")
print("=" * 100)
print(results_df.sort_values('calinski_harabasz', ascending=False).head(10))

## 4. Visualize Results

Create plots to help identify the optimal k value.

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Elbow Method (Inertia)
for init in init_methods:
    data = results_df[results_df['init'] == init]
    axes[0, 0].plot(data['k'], data['inertia'], marker='o', label=f"init={init}")
axes[0, 0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0, 0].set_ylabel('Inertia (Within-cluster Sum of Squares)', fontsize=12)
axes[0, 0].set_title('Elbow Method: Find the "Elbow" Point', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Silhouette Score
for init in init_methods:
    data = results_df[results_df['init'] == init]
    axes[0, 1].plot(data['k'], data['silhouette'], marker='o', label=f"init={init}")
axes[0, 1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[0, 1].set_ylabel('Silhouette Score', fontsize=12)
axes[0, 1].set_title('Silhouette Score (Higher is Better)', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.3)

# Plot 3: Davies-Bouldin Index
for init in init_methods:
    data = results_df[results_df['init'] == init]
    axes[1, 0].plot(data['k'], data['davies_bouldin'], marker='o', label=f"init={init}")
axes[1, 0].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1, 0].set_ylabel('Davies-Bouldin Index', fontsize=12)
axes[1, 0].set_title('Davies-Bouldin Index (Lower is Better)', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Calinski-Harabasz Score
for init in init_methods:
    data = results_df[results_df['init'] == init]
    axes[1, 1].plot(data['k'], data['calinski_harabasz'], marker='o', label=f"init={init}")
axes[1, 1].set_xlabel('Number of Clusters (k)', fontsize=12)
axes[1, 1].set_ylabel('Calinski-Harabasz Score', fontsize=12)
axes[1, 1].set_title('Calinski-Harabasz Score (Higher is Better)', fontsize=14, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('kmeans_evaluation_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Plots saved as 'kmeans_evaluation_metrics.png'")

In [ ]:
# Create a combined score ranking
# Normalize metrics to [0, 1] scale and combine
results_norm = results_df.copy()

# Normalize: higher silhouette is better, lower davies_bouldin is better, higher calinski is better
results_norm['silhouette_norm'] = (results_norm['silhouette'] - results_norm['silhouette'].min()) / \
                                   (results_norm['silhouette'].max() - results_norm['silhouette'].min())

results_norm['davies_bouldin_norm'] = 1 - ((results_norm['davies_bouldin'] - results_norm['davies_bouldin'].min()) / \
                                             (results_norm['davies_bouldin'].max() - results_norm['davies_bouldin'].min()))

results_norm['calinski_norm'] = (results_norm['calinski_harabasz'] - results_norm['calinski_harabasz'].min()) / \
                                 (results_norm['calinski_harabasz'].max() - results_norm['calinski_harabasz'].min())

# Combined score (equal weights)
results_norm['combined_score'] = (results_norm['silhouette_norm'] + 
                                   results_norm['davies_bouldin_norm'] + 
                                   results_norm['calinski_norm']) / 3

print("=" * 100)
print("TOP 10 CONFIGURATIONS BY COMBINED SCORE")
print("=" * 100)
print(results_norm.sort_values('combined_score', ascending=False)[[
    'k', 'init', 'combined_score', 'silhouette', 'davies_bouldin', 'calinski_harabasz'
]].head(10))

## 5. Select Optimal K and Fit Final Model

Based on the evaluation metrics above, select the optimal k value.

In [ ]:
# Get best configuration
best_config = results_norm.sort_values('combined_score', ascending=False).iloc[0]
optimal_k = int(best_config['k'])
optimal_init = best_config['init']

print(f"🏆 RECOMMENDED CONFIGURATION:")
print(f"   k = {optimal_k}")
print(f"   init = '{optimal_init}'")
print(f"   Combined Score = {best_config['combined_score']:.4f}")
print(f"\n📊 Metrics:")
print(f"   Silhouette Score: {best_config['silhouette']:.4f}")
print(f"   Davies-Bouldin Index: {best_config['davies_bouldin']:.4f}")
print(f"   Calinski-Harabasz Score: {best_config['calinski_harabasz']:.2f}")

In [ ]:
# Allow manual override if desired
# OVERRIDE: Set to None to use recommended k, or set to desired value (e.g., 10)
MANUAL_K = None  # Change to 10 if you want to force k=10 for easier labeling

if MANUAL_K is not None:
    final_k = MANUAL_K
    print(f"⚠️  MANUAL OVERRIDE: Using k = {final_k} instead of recommended k = {optimal_k}")
else:
    final_k = optimal_k
    print(f"✅ Using recommended k = {final_k}")

In [ ]:
# Fit final k-means model
print(f"\n⏳ Fitting final k-means model with k={final_k}...")

kmeans_final = KMeans(
    n_clusters=final_k,
    init='k-means++',  # Generally more reliable
    n_init=50,  # More runs for final model
    max_iter=300,
    random_state=42
)

cluster_labels = kmeans_final.fit_predict(X_scaled)

# Add cluster labels to dataframe
df['cluster'] = cluster_labels

print(f"✅ K-means clustering complete!")
print(f"\n📊 Cluster distribution:")
print(df['cluster'].value_counts().sort_index())

## 6. Cluster Profiling

Analyze each cluster to understand its characteristics and enable manual labeling.

In [ ]:
# Calculate cluster statistics for key features
key_features = ['distance_m', 'ascent_m', 'duration_s', 'Turn_Density', 
                'on_road', 'off_road', 'Flat Section', 'Steep Section']

cluster_profiles = df.groupby('cluster')[key_features].agg(['mean', 'median', 'std']).round(2)

print("=" * 120)
print("CLUSTER PROFILES")
print("=" * 120)
print(cluster_profiles)

In [ ]:
# Create detailed cluster summary
for cluster_id in range(final_k):
    cluster_data = df[df['cluster'] == cluster_id]
    
    print("\n" + "=" * 100)
    print(f"CLUSTER {cluster_id} - {len(cluster_data):,} routes ({len(cluster_data)/len(df)*100:.1f}%)")
    print("=" * 100)
    
    # Distance characteristics
    print(f"\n📏 Distance:")
    print(f"   Mean: {cluster_data['distance_m'].mean()/1000:.1f} km")
    print(f"   Median: {cluster_data['distance_m'].median()/1000:.1f} km")
    print(f"   Range: {cluster_data['distance_m'].min()/1000:.1f} - {cluster_data['distance_m'].max()/1000:.1f} km")
    
    # Elevation characteristics
    print(f"\n⛰️  Elevation:")
    print(f"   Mean ascent: {cluster_data['ascent_m'].mean():.0f} m")
    print(f"   Median ascent: {cluster_data['ascent_m'].median():.0f} m")
    
    # Surface type (dominant)
    print(f"\n🛣️  Surface:")
    print(f"   On-road: {cluster_data['on_road'].mean():.1f}%")
    print(f"   Off-road: {cluster_data['off_road'].mean():.1f}%")
    print(f"   Paved Road: {cluster_data['Paved_Road'].mean():.1f}%")
    print(f"   Gravel: {cluster_data['Gravel_Tracks'].mean():.1f}%")
    
    # Gradient profile
    print(f"\n📐 Gradient:")
    print(f"   Flat: {cluster_data['Flat Section'].mean():.1f}%")
    print(f"   Moderate: {cluster_data['Moderate Section'].mean():.1f}%")
    print(f"   Steep: {cluster_data['Steep Section'].mean():.1f}%")
    
    # Example routes
    print(f"\n📋 Sample routes:")
    sample_routes = cluster_data.head(5)[['id', 'name', 'distance_m', 'ascent_m']]
    for idx, route in sample_routes.iterrows():
        print(f"   • {route['name'][:60]} (ID: {route['id']}, {route['distance_m']/1000:.1f}km, {route['ascent_m']:.0f}m ascent)")

## 7. Visualize Cluster Characteristics

In [ ]:
# Create cluster visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Distance vs Ascent by Cluster
for cluster_id in range(final_k):
    cluster_data = df[df['cluster'] == cluster_id]
    axes[0, 0].scatter(
        cluster_data['distance_m'] / 1000, 
        cluster_data['ascent_m'],
        label=f'Cluster {cluster_id}',
        alpha=0.5,
        s=10
    )
axes[0, 0].set_xlabel('Distance (km)', fontsize=12)
axes[0, 0].set_ylabel('Ascent (m)', fontsize=12)
axes[0, 0].set_title('Clusters: Distance vs Elevation', fontsize=14, fontweight='bold')
axes[0, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Cluster size distribution
cluster_sizes = df['cluster'].value_counts().sort_index()
axes[0, 1].bar(cluster_sizes.index, cluster_sizes.values, color=sns.color_palette("husl", final_k))
axes[0, 1].set_xlabel('Cluster ID', fontsize=12)
axes[0, 1].set_ylabel('Number of Routes', fontsize=12)
axes[0, 1].set_title('Cluster Size Distribution', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Plot 3: Average distance by cluster
avg_distance = df.groupby('cluster')['distance_m'].mean() / 1000
axes[1, 0].bar(avg_distance.index, avg_distance.values, color=sns.color_palette("husl", final_k))
axes[1, 0].set_xlabel('Cluster ID', fontsize=12)
axes[1, 0].set_ylabel('Average Distance (km)', fontsize=12)
axes[1, 0].set_title('Average Distance by Cluster', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Plot 4: On-road vs Off-road by cluster
surface_data = df.groupby('cluster')[['on_road', 'off_road']].mean()
x = np.arange(final_k)
width = 0.35
axes[1, 1].bar(x - width/2, surface_data['on_road'], width, label='On-road', alpha=0.8)
axes[1, 1].bar(x + width/2, surface_data['off_road'], width, label='Off-road', alpha=0.8)
axes[1, 1].set_xlabel('Cluster ID', fontsize=12)
axes[1, 1].set_ylabel('Percentage (%)', fontsize=12)
axes[1, 1].set_title('Surface Type by Cluster', fontsize=14, fontweight='bold')
axes[1, 1].set_xticks(x)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('cluster_characteristics.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Cluster visualization saved as 'cluster_characteristics.png'")

## 8. Save Model and Cluster Labels

Save the trained k-means model for production use.

In [ ]:
# Save k-means model
joblib.dump(kmeans_final, 'kmeans.pkl')
print(f"✅ K-means model saved as 'kmeans.pkl'")
print(f"   Clusters: {final_k}")
print(f"   Inertia: {kmeans_final.inertia_:.2f}")

# Save cluster-labeled data
df.to_csv('Data_Engineered_with_clusters.csv', index=False)
print(f"\n✅ Data with cluster labels saved as 'Data_Engineered_with_clusters.csv'")

# Save cluster profiles for reference
cluster_profiles.to_csv('cluster_profiles.csv')
print(f"✅ Cluster profiles saved as 'cluster_profiles.csv'")

## 9. Next Steps: Manual Labeling

Based on the cluster profiles above, manually assign descriptive labels to each cluster.

**Example labels:**
- Cluster 0: "Short flat city rides"
- Cluster 1: "Long mountain climbs"
- Cluster 2: "Moderate suburban routes"
- etc.

Create a mapping dictionary that can be used in production:

```python
CLUSTER_LABELS = {
    0: "Short flat city rides",
    1: "Long mountain climbs",
    # ... add more
}
```

**For curveball recommendations:**
1. User uploads GPX → extract features
2. Predict cluster: `user_cluster = kmeans.predict(scaled_features)`
3. Get 5 KNN recommendations (existing flow)
4. Get 1 curveball from different cluster: 
   - Filter routes where `cluster != user_cluster`
   - Find nearest route from that filtered set

## Summary

**Outputs:**
- `kmeans.pkl`: Trained k-means model
- `Data_Engineered_with_clusters.csv`: Full dataset with cluster labels
- `cluster_profiles.csv`: Statistical profiles of each cluster
- `kmeans_evaluation_metrics.png`: Metric comparison plots
- `cluster_characteristics.png`: Cluster visualization plots

**Files to deploy:**
- `kmeans.pkl` → Copy to `cycle_more/api_faf/app/`
- Cluster labels dictionary → Add to service code